# Automating building a test openvswitch

## Getting started

The simplest possible way to automate deploying an experiment in Chameleon is to use a Jupyter document. Main reason is that you can run (shell) commands from within the document and see the output. This document goes in more depth than [Getting Started Guide][1], namely it shows how to
1. Create a lease
1. Create ssh keypair
1. Get a floating IP
2. Create the required baremetal/virtual servers
1. Access the baremetal server
1. Build the test environment in the server. In this case a Open vSwitch lab.
1. Run experiment to our hearts content
1. Display results
1. Destroy the servers
1. Release floating IP
1. Delete SSH keypair
1. Destroy the lease.

from the command line.

### Limitations

1. We can only run non-interactive commands here

### Conventions

1. Some of the resources will be associated with the user who is running this doc, in this case `$USER`.
1. If you are only a member of one project, you can skip this step, as the project will be selected for you by default. Otherwise, take a look at the current value of the environmental variable `OS_PROJECT_NAME`:

[1]: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html

In [ ]:
echo $OS_PROJECT_NAME

to ensure it matches the name of the project you want to run this lab on. If it does not, change it (uncommend the `export OS_PROJECT_NAME` line and set `your-project` to match the project name).

**NOTE:** If project has a nickname, you **must** use it instead.

In [ ]:
# Set up user's project (Replace 'your-project' with your project name)
# export OS_PROJECT_NAME='your-project'
echo "New project name = $OS_PROJECT_NAME"

**NOTE:** Just to be on the safe side, let's test it (we will explain the command later on):

In [ ]:
openstack keypair list

If the output looks like this:
```
+------------+-------------------------------------------------+
| Name       | Fingerprint                                     |
+------------+-------------------------------------------------+
| defaultkey | d0:89:5b:61:6a:64:dd:c8:db:67:32:32:45:71:b0:b8 |
+------------+-------------------------------------------------+
```
you can continue to the next step. If it looks like this:
```
The request you have made requires authentication. (HTTP 401) (Request-ID: req-76ad404f-0043-45e9-84cf-0504843888ab)
```
figure out what is going on before continuing. One thing to check is whether the project name you provide works (the nickname issue mentioned before is one possible reason).

You can also set the site you want to use via the `OS_REGION_NAME` setting; this defaults to `CHI@UC`.

In [ ]:
# Set region (Optional, default to 'CHI@UC')
# export OS_REGION_NAME='CHI@UC'
echo $OS_REGION_NAME

### Define some variables
We need to define the names for our
  1. Lease
  1. Server
  1. Private network
  1. Public network
  1. SSH key. By default Jupyter places the user's ssh keys in `~/work/.ssh`, but that should not stop us from placing them wherever we need them. Also, for this example, we delete the keys as part of the tearing down procedure.
  1. Type for the instance/node/servers
  1. How many nodes we will need. Always get a least an extra one since one of the servers we want to build might be assigned to a flakey physical host. If we have spares, openstack will simply drop it and go to the next one.


In [ ]:
export LEASE_NAME="$USER-ovs-test"
export SERVER_NAME="$USER-ovs-server"
export PRIVATE_NETWORK_NAME="sharednet1"
export PUBLIC_NETWORK_NAME="public"
export SSHKEY_FILE="$HOME/.ssh/$USER-chameleon"
export SSHKEY_NAME="ChameleonKey"
export NODE_TYPE="compute_haswell"
export MAX_SERVERS=2

These environmental variables only exist within the scope of this Jupyter document.

### Create a lease 
Specifically, we will create the lease `$LEASE_NAME`. 

In [ ]:
blazar lease-create --physical-reservation \
   min=1,"max=$MAX_SERVERS",resource_properties='["=", "$node_type", "'"$NODE_TYPE"'"]' "$LEASE_NAME"

Before we continue let's verify if the lease was successful created. This might take a few minutes, or just crash horribly. Since we are automating this, we need to account for these options.

In [ ]:
lease_status=""

# Lease in a sorry state
while [[ $lease_status == "TERMIN"* ]] || [[ $lease_status == "ERROR" ]] 
do
   echo "Lease it is in a sorry state. Restarting it."
   # Delete old lease
   blazar lease-delete "$LEASE_NAME"
   blazar lease-create --physical-reservation \
      min=1,"max=$MAX_SERVERS",resource_properties='["=", "$node_type", "$NODE_TYPE"]' "$LEASE_NAME"
   lease_status=$(blazar lease-show --format value -c status "$LEASE_NAME")
done
echo "Lease creation successfuly started."

# Now wait for lease to be ready before going to the next step
while [[ $lease_status != "ACTIVE" ]]
do
   sleep 5
   lease_status=$(blazar lease-show --format value -c status "$LEASE_NAME")
done

echo "Lease $LEASE_NAME ready for business"

Wait until seeing the `Lease $LEASE_NAME ready for business` message before continuing.

Another way is to keep track of PID and wait until it is done.

Now, some commands can use this lease name but others need a lease ID instead. So, while we are here we might as well get the `$lease_id`.

In [ ]:
lease_id=$(blazar lease-show  --format value -c  reservations "$LEASE_NAME" |grep \"id\"| cut -d \" -f4)

#### Get the ID of the network we will create the baremetal server on
In the previous step we obtained `lease_id`, the ID of the lease named `$USER-default-lease`. Now we will do the same but for the network we will run our server in. Use the `sharednet1` network unless you have a good reason not to. Further information on this network is [available in the docs][1].

[1]: https://chameleoncloud.readthedocs.io/en/latest/technical/networks/networks_basic.html#shared-network

In [ ]:
# Get the network ID associated with sharednet1
network_id=$(openstack network show --format value -c id $PRIVATE_NETWORK_NAME)

### Create a SSH key pair
One of the goals for this document is to access the the baremetal server; that will be achieved by using ssh to connect to the server. For [security][1], servers created in Chameleon are by default accessed using SSH key pair authentication. 

Openstack can store the public key, or keys, which can then be passed to the instance. To see which keys are currently defined you can type

[1]:https://docs.openstack.org/horizon/latest/user/configure-access-and-security-for-instances.html

In [ ]:
openstack keypair list

We will need to create a key pair, say, `$SSHKEY_NAME` in the RSA format with a size of `4096` bits (the minimum size to use nowadays) and saved as `$SSHKEY_FILE` for the private key and `$SSHKEY_FILE.pub` for the public. 

**NOTE:** By default we do not have a `~/.ssh` dir here, so we need to create one first.

In [ ]:
[ -d ~/.ssh ] || mkdir -m 0700 ~/.ssh
ssh-keygen -t rsa -b 4096  -P '' -C $SSHKEY_NAME -f $SSHKEY_FILE

We are cheating by using `echo "yes"` to say we do not want to use a passphrase associated with this key pair. If you choose to use a passphrase, remove everything before `ssh-keygen`.

Next is to add it to the list of keypairs openstack know to be associated with your account. In reality it is just uploading the public key, which is what you really need to ssh into a host. As with the lease, we do need a [name][2] associated with this key pair:

[2]:https://docs.openstack.org/python-openstackclient/latest/cli/command-objects/keypair.html

In [ ]:
openstack keypair create --public-key $SSHKEY_FILE.pub  $SSHKEY_NAME
openstack keypair list

### Get Floating IP

By default, the server will only have a private IP assigned (in this case, an IP in the `PRIVATE_NETWORK_NAME` network). In order to connect (using SSH or other protocol) to the server from your desktop or another computer in the internet, you should assign a [public-facing floating IP][1]. There are a limited amount of public IPs available across the entire Chameleon testbed, so try to keep the amount of nodes with a public IP to a minimum! A common practice is to set up one node as a "login node" with a public IP, and logging in to that node to manage all of your project's nodes.

[1]: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html#associating-an-ip-address

In [ ]:
# Request a public floating IP (in the 'public' network)
server_ip=$(openstack floating ip create public --format value -c floating_ip_address)
echo "Public IP for this lab is $server_ip"

Later on we will assign `$server_ip` to the right instance.

#### Should I have more than one floating (public) IP?
The short answer is **no**. Long answer is that 

1. There are very few times when someone needs more than one public IP as opposite to having one instance you can remote in and then go to others. For instance, you could use port forwarding to access all the instances in your experiment directly.
1. There is a finite number if static IPs. By using more than one, someone else may end up with none. This is also the reason why once you finish your lab, you should immediately release the allocated floating IP.

### Create the required baremetal servers

Let's launch a Centos 7 baremetal server called `$SERVER_NAME` on network `$PRIVATE_NETWORK_NAME` (identified using `$network_id`) and lease `$LEASE_NAME` (identified using `$lease_id`). It will be accessible using the ssh keypair `$SSHKEY_NAME` we created earlier.

In [ ]:
openstack server create \
--flavor baremetal \
--image CC-CentOS7 \
--nic net-id="$network_id" \
--hint reservation="$lease_id" \
--key-name="$SSHKEY_NAME" \
--wait \
$SERVER_NAME

If we need more than one host, use a loop. We can then call the servers `$SERVER_NAME-1`, `$SERVER_NAME-2` and so on or give them more functional names like `$SERVER_NAME-webserver` and `$SERVER_NAME-database`. It all depends on what you want to do with them and how.

``` bash
for i in 
do
  openstack server create \
  --flavor baremetal \
  --image CC-CentOS7 \
  --nic net-id="$network_id" \
  --hint reservation="$lease_id" \
  --key-name="$SSHKEY_NAME" \
  --wait \
  "$SERVER_NAME-$i"
done
```

#### But wait! And then wait some more!

Creating a server (or node) is not an instantanous process specially if it is a baremetal node. Chameleon has to boot the node, install the OS, move it to the right network, and then it is ready to receive the public IP. All of these steps can take **up to 10 minutes**, which is why we suggest to script building these nodes: you can let the script monitor the status of the node and then let you know when it is ready to be used. Or, same script can start using said node on its own. This is the reason why we passed the `--wait` flag when we created the server in the previous step: it will not allow the script to continue until the server is ready for the next step (adding the public IP).

#### Steps:

1. Only continue when the server status = `ACTIVE`. `BUILD != ACTIVE` 
1. Get a public floating IP. Either you already have a list of IPs you can use with your project or you will need to [request an IP][2]. For this example we will do the later. You need to know the name of the public network, which in this case is 'public'.
1. Assign the IP to the server we created earlier.
1. Ensure that there is a firewall rule or security policy allowing you to connect to server on port 22 (ssh). This is the default.
1. Set up a remote connection to the node.

[2]: https://docs.openstack.org/python-openstackclient/latest/cli/command-objects/floating-ip.html#floating-ip-create


In [ ]:
# Wait until server is ready before continuing
server_status=""
while [[ $server_status != "ACTIVE" ]]
do
   sleep 5
   server_status=$(openstack server list --format value -c Status --name "$SERVER_NAME")
done

# Assign a public floating IP to $SERVER_NAME
openstack server add floating ip "$SERVER_NAME" "$server_ip"

## Access the baremetal server(s)

### Testing the connection
Server has now a publicly facing IP. Let's see if it works (assuming you have netcat installed. If you are running this Jupyter book, you have netcat).

In [ ]:
# Check if we can connect to server on port 22.
ssh_status=""
while [[ $ssh_status != "Up" ]]
do
   sleep 120
   ssh_status=$(nc -q 0 -w 1 "${server_ip}" 22 < /dev/null &> /dev/null && echo "Up" || echo "Down")
done

The above script returns "Up" if it can connect to port 22 (SSH) on our recently 
created server using the public IP, otherwise it returns "Down". Of course, the answer can be changed so it is is more script-friendly.

### Logging into the server we just created

We ssh using the private key to verify it works. The default username for Chameleon-build images is `cc`. While logged in, might as well take a quick look; remember this server will be wiped once we are done.

In [ ]:
server_ip=192.5.87.142
moo="ssh -o \"StrictHostKeyChecking no\" -i $SSHKEY_FILE cc@$server_ip"
eval "$moo" pwd 

## Experiment time
### Installing Open vSwitch
We will install it from scratch because we do not want to mess with any running network. 

First we need to get the prereq packages

In [ ]:
eval "$moo" /bin/bash << EOF
sudo yum install -y python3-pip bison byacc cscope \
 ctags diffstat flex gcc gcc-c++ gettext git indent intltool libtool \
 patch patchutils rcs 
sudo pip3 install six
EOF

Now get a copy of Open vSwitch

In [ ]:
eval "$moo" git clone https://github.com/openvswitch/ovs.git

Next, we build it. This will generate a lot of output, and is one of the reasons that building a lab using Jupyter works only to a certain point. Beyond that you may want to consider something like chef, puppet, or ansible.

In [ ]:
eval "$moo" /bin/bash << EOF
cd ovs
./boot.sh
./configure
make -j4
EOF

Finally we run the sandbox

In [ ]:
make sandbox

To exit the sandbox type exit or Control+D.

## Cleaning up after ourselves
As the last task in this document, tear everything down. We can put it all back together by running this jupyter book again later.

In [ ]:
# Delete server. If you have more than one server in this project, 
# you need to loop over them all somehow.
openstack server delete $SERVER_NAME && \

# Delete the public facing IP
openstack floating ip delete $server_ip && \

# Delete lease
blazar lease-delete $LEASE_NAME && \

# Delete key (files must be deleted manually)
openstack keypair delete $SSHKEY_NAME
rm $SSHKEY_FILE $SSHKEY_FILE.pub